In [1]:
import requests
import pandas as pd
import json
import duckdb

In [2]:
r = requests.get("https://data.caf.fr/api/explore/v2.1/catalog/datasets/txcouv_pe_epci/records?refine=annee%3A%222023%22")

In [3]:
data_pe = r.json()

In [4]:
data_pe.keys()

dict_keys(['total_count', 'results'])

In [5]:
data_pe['results'][0].keys()

dict_keys(['annee', 'numepci', 'nomepci', 'txcouv_psu_col_epci', 'txcouv_hors_psu_col_epci', 'txcouv_eaje_epci', 'txcouv_prescol_epci', 'txcouv_am_ind_epci', 'txcouv_gad_ind_epci', 'txcouv_ind_epci', 'txcouv_epci'])

In [6]:
df_pe = pd.json_normalize(data_pe['results'])

In [7]:
df_pe.shape

(10, 11)

In [8]:
query = """ 
SELECT 
    numepci as siren,
    txcouv_epci
FROM df_pe
WHERE txcouv_epci IS NOT NULL
ORDER BY numepci
"""

df_pe_final = duckdb.sql(query)

In [31]:
df_pe_final

┌───────────┬─────────────┐
│   siren   │ txcouv_epci │
│  varchar  │   double    │
├───────────┼─────────────┤
│ 200000438 │        76.3 │
│ 200035723 │        56.2 │
│ 200043065 │        65.8 │
│ 200066223 │        44.0 │
│ 200067122 │        45.3 │
│ 200069912 │        58.6 │
│ 200070977 │        51.0 │
│ 200090579 │        71.0 │
│ 240700864 │        65.0 │
│ 248500258 │        86.2 │
├───────────┴─────────────┤
│ 10 rows       2 columns │
└─────────────────────────┘

In [12]:
df_epci = pd.read_csv("../data/data_pe/raw/data_epci.csv",encoding= 'latin1',sep=';')
df_epci

,dept,siren,raison_sociale,nature_juridique,mode_financ,nb_membres,total_pop_tot,total_pop_mun,dep_com,insee,siren_membre,nom_membre,ptot_2025,pmun_2025
0,01,200029999.0,CC Rives de l'Ain - Pays du Cerdon,CC,FPU,14.0,15 156,14 873,01,01304,210103040.0,Pont-d'Ain,2 912,2 862
1,01,200029999.0,CC Rives de l'Ain - Pays du Cerdon,CC,FPU,14.0,15 156,14 873,01,01199,210101994.0,Jujurieux,2 246,2 209
2,01,200029999.0,CC Rives de l'Ain - Pays du Cerdon,CC,FPU,14.0,15 156,14 873,01,01363,210103636.0,Saint-Jean-le-Vieux,1 873,1 799
3,01,200029999.0,CC Rives de l'Ain - Pays du Cerdon,CC,FPU,14.0,15 156,14 873,01,01314,210103149.0,Priay,1 826,1 803
4,01,200029999.0,CC Rives de l'Ain - Pays du Cerdon,CC,FPU,14.0,15 156,14 873,01,01273,210102737.0,Neuville-sur-Ain,1 823,1 798
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34867,2B,242020105.0,CC de Calvi Balagne,CC,FPU,14.0,12 842,12 638,2B,2B138,212001382.0,Lavatoggio,158,156
34868,2B,242020105.0,CC de Calvi Balagne,CC,FPU,14.0,12 842,12 638,2B,2B296,212002968.0,Sant'Antonino,133,132
34869,2B,242020105.0,CC de Calvi Balagne,CC,FPU,14.0,12 842,12 638,2B,2B153,212001531.0,Manso,120,117
34870,2B,242020105.0,CC de Calvi Balagne,CC,FPU,14.0,12 842,12 638,2B,2B025,212000251.0,Avapessa,81,81


In [13]:
query_epci = """ 
SELECT DISTINCT siren as id_epci,
raison_sociale as nom_epci,
dept
FROM df_epci
"""

df_epci_final = duckdb.sql(query_epci)
df_epci_final

┌─────────────┬─────────────────────────────────────────────────┬─────────┐
│   id_epci   │                    nom_epci                     │  dept   │
│   double    │                     varchar                     │ varchar │
├─────────────┼─────────────────────────────────────────────────┼─────────┤
│ 200040350.0 │ CC Bugey Sud                                    │ 01      │
│ 200042497.0 │ CC Dombes Saône Vallée                          │ 01      │
│ 200071371.0 │ CC Bresse et Saône                              │ 01      │
│ 200071751.0 │ CA du Bassin de Bourg-en-Bresse                 │ 01      │
│ 240100610.0 │ CC de la Côtière à Montluel                     │ 01      │
│ 200043495.0 │ CA du Pays de Laon                              │ 02      │
│ 200071769.0 │ CC Picardie des Châteaux                        │ 02      │
│ 200071983.0 │ CC Thiérache Sambre et Oise                     │ 02      │
│ 240200444.0 │ CC de la Thiérache du Centre                    │ 02      │
│ 240200493.

In [17]:
query = """
        SELECT 
            s1.dept,
            s1.id_epci,
            s1.nom_epci,
            'i130' AS id_indicator,
            s2.txcouv_epci as valeur_brute   
        FROM df_epci_final as s1
        LEFT JOIN df_pe_final as s2
        ON s1.id_epci = s2.siren
        ORDER BY s1.dept, s1.id_epci
        """
    
df_complete = duckdb.sql(query)
df_complete


┌─────────┬─────────────┬─────────────────────────────────────────────────┬──────────────┬──────────────┐
│  dept   │   id_epci   │                    nom_epci                     │ id_indicator │ valeur_brute │
│ varchar │   double    │                     varchar                     │   varchar    │    double    │
├─────────┼─────────────┼─────────────────────────────────────────────────┼──────────────┼──────────────┤
│ 01      │ 200029999.0 │ CC Rives de l'Ain - Pays du Cerdon              │ i130         │         NULL │
│ 01      │ 200040350.0 │ CC Bugey Sud                                    │ i130         │         NULL │
│ 01      │ 200042497.0 │ CC Dombes Saône Vallée                          │ i130         │         NULL │
│ 01      │ 200042935.0 │ CA Haut-Bugey Agglomération                     │ i130         │         NULL │
│ 01      │ 200069193.0 │ CC de la Dombes                                 │ i130         │         NULL │
│ 01      │ 200070118.0 │ CC Val de Saône Cent